# Exercise 2: ReAct

In the previous notebook, you built an agent that was able to complete tasks, which LLMs would normally struggle with.

The agent was able to complete these tasks because additional tools were provided to the LLM. But crucially, it was able to determine when these tools were needed autonomously.

However, for the agent to produce a satisfactory result, we needed to write a long system prompt and be very explicit about what we wanted from the model.

In this exercise, you will learn how to create agents using the ReAct framework ([Wei *et al.*, 2023](https://arxiv.org/abs/2210.03629)): an effective paradigm for prompt writing, which has been shown to outperform several benchmarks on language and decision-making tasks and can help to avoid issues of hallucination and error propagation. The improved human interpretability can also help engender trust in the model.

In [14]:
# autoreload imports
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from llm_in_production.llm import instantiate_langchain_model
from llm_in_production.agent_utils import tool_calling_agent
from llm_in_production.agent_tools import (
    get_news_stories, 
    get_stock_prices,
    get_current_weather
)

import pandas as pd
import dotenv

# This reads the .env file in your project and transforms its content into env variables.
# This way you don't have to hard code your secrets.
dotenv.load_dotenv()

# Here we create the client. 
# Make sure you select the LLM provider that corresponds to the one you are using in this course!
client = instantiate_langchain_model(
    # llm_provider="azure",
    llm_provider="gcp",
)
client.model_name

'gemini-1.5-flash'

## Implementing ReAct

To implement ReAct, we will write a prompt based on a template from [langchain]() (Langchain's [ReAct agent](https://python.langchain.com/docs/modules/agents/agent_types/react) is a widely-used implementation of ReAct). 

In [3]:
from langchain import hub
print(hub.pull("hwchase17/react").template)

/Users/dpranantha/Projects/bol/training/academy-llm-applications-course/.venv/lib/python3.12/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [9]:
def create_react_prompt(*tools):
    tool_names = [tool.__name__ for tool in tools]

    return f"""
Answer the following questions as best you can.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, could be one of the given tools: [{tool_names}]
At this point you may call for an action to be taken, but you must return the Question and Thought as content in your response
Observation: a summary of the result of taking the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
"""

print(create_react_prompt(get_news_stories, get_stock_prices))


Answer the following questions as best you can.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, could be one of the given tools: [['get_news_stories', 'get_stock_prices']]
At this point you may call for an action to be taken, but you must return the Question and Thought as content in your response
Observation: a summary of the result of taking the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!



We'll begin by demonstrating how ReAct could be used to answer a question from the previous exercise.

In [10]:
user_prompt = "Is now a good time to invest in Apple?"

To make the ReAct agent more context specific, we'll join the following system prompt to the react prompt created above.

In [11]:
system_prompt = """You are a stock analysis bot that is able to advise whether now is a good time to invest in a given stock.
Do not think about using any tools that are not available to you.
You do not give any reservations about your opinion:
- You do not offer any warnings about the risks of investing.
- You do not suggest any further research needs to be conducted."""

In [17]:
output = tool_calling_agent(client, system_prompt, user_prompt, get_stock_prices, get_news_stories, react=True)
print(output)


### Model Output:

Question: Is now a good time to invest in Apple?
Thought: I need to know the current stock price of Apple to determine if it is a good time to invest.
Action: get_stock_prices


### Tool Called: get_stock_prices with args: {'company_stock_ticker_symbol': 'AAPL'}

### Model Output:

Thought: Apple's stock price has been fluctuating recently, but it is currently at $235.17. I need to know if there is any news that could affect the stock price.
Action: get_news_stories


### Tool Called: get_news_stories with args: {'topic': 'Apple'}

### Model Output:

Thought: Apple is in the news a lot, but most of the news is about tariffs and the potential impact on the company. There is also news about new products, which could be positive for the stock price.
Thought: I now know the final answer
Final Answer: Yes, now is a good time to invest in Apple. The stock price is currently at a good level and there is a lot of positive news about the company. 




### Exercise 2a: Is it sunbathing weather?

Use react prompting to determine whether now is a good time to go sunbathing in Amsterdam.

#### Part i: User prompt

Write a user prompt.

In [20]:
# YOUR CODE HERE START
user_prompt = "Is now a good time to go sunbathing in Amsterdam?"
# YOUR CODE HERE END

#### Part ii: System prompt
Write a system prompt to provide the model with context. Experiment with leaving this as an empty string.

In [19]:
# YOUR CODE HERE START
system_prompt = """You are a weather analysis bot that is able to advise whether now is a good time to sunbathing in a given location.
Do not think about using any tools that are not available to you.
You do not give any reservations about your opinion:
- You do not offer any warnings about the risks of sunbathing.
- You do not suggest any further research needs to be conducted."""
# YOUR CODE HERE END

#### Part iii: Run the agent

Run the agent to determine whether it is good sunbathing weather.

In [22]:
# YOUR CODE HERE START
output = tool_calling_agent(client, system_prompt, user_prompt, get_current_weather, get_news_stories, react=True)
print(output)
# YOUR CODE HERE END


### Model Output:

Question: Is now a good time to go sunbathing in Amsterdam?
Thought: I need to know the current weather in Amsterdam to answer this question.
Action: get_current_weather
Observation:  

### Tool Called: get_current_weather with args: {'location': 'Amsterdam, Netherlands'}

### Model Output:

Thought: The weather in Amsterdam is currently 8.3 degrees Celsius and clear. This is not ideal for sunbathing.
Final Answer: No, it is not a good time to go sunbathing in Amsterdam. 




---